# Cohere Embed 4

This Jupyter Notebook demonstrates the multimodal and multilingual capabilities of Cohere Embed 4 model on Amazon Bedrock. The notebook showcases three key features in sections 1 to 3: text-to-vector embedding conversion, image-to-vector embedding conversion, and mixed-modality input that combines both text and images in a single request. 

Sections 4 to 6 show the creation of a S3 Vector Bucket and Index to store the generated embeddings. The S3 Vectors API is used to query the embeddings.


## Section 1) Using Embed 4 to generate embeddings for text input

In [1]:
import boto3
import json

# Example text
text1 = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
text2 = "It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

# Initialize Bedrock Runtime client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')
# bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')

# Request body
body = {
    "texts": [text1, text2],
    "input_type": "search_document",
    "embedding_types": ["float"]
}

# Use inference profile instead of direct model ID for us-west-2
inference_profile_id = 'us.cohere.embed-v4:0'
# Use model ID for us-east-1
# model_id = 'cohere.embed-v4:0'

response = bedrock_runtime.invoke_model(
    modelId=inference_profile_id,
    # modelId=model_id,
    body=json.dumps(body),
    accept='*/*',
    contentType='application/json'
)

# Parse response
result = json.loads(response['body'].read())
# Uncomment this print statement to print out the entire result
# print(json.dumps(result, indent=2))

# Parse response and limit output to first 30 lines
output = json.dumps(result, indent=2)
lines = output.split('\n')
if len(lines) > 100:
    print('\n'.join(lines[:30]))
    print(f'... (truncated, showing first 100 of {len(lines)} lines)')
else:
    print(output)

{
  "id": "56892f1a-19dd-4327-9991-819eb7c13d8b",
  "texts": [
    "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.",
    "It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."
  ],
  "embeddings": {
    "float": [
      [
        -0.07324219,
        -0.06591797,
        -0.059570312,
        -0.009887695,
        -0.02355957,
        0.009216309,
        0.0037994385,
        0.045166016,
        -0.021240234,
        -0.02758789,
        -0.017089844,
        0.017944336,
        -0.043701172,
        -0.012268066,
  

## Section 2) Using Embed 4 to generate embeddings for image input

In [2]:
import base64

# Load image from https://s2.q4cdn.com/299287126/files/doc_financials/2025/q2/Webslides_Q225.pdf
with open('2025q2.png', 'rb') as f:
    image_b64 = base64.b64encode(f.read()).decode('utf-8')

# Request - image only
body = {
    "images": [f"data:image/jpeg;base64,{image_b64}"],
    "input_type": "search_document",
    "embedding_types": ["float"]
}

# Call API
client = boto3.client('bedrock-runtime', region_name='us-west-2')
response = client.invoke_model(
    modelId=inference_profile_id,
    # modelId='cohere.embed-v4:0',
    body=json.dumps(body),
    contentType='application/json'
)

result = json.loads(response['body'].read())
# print(json.dumps(result, indent=2))

# Parse response and limit output to first 30 lines
output = json.dumps(result, indent=2)
lines = output.split('\n')
if len(lines) > 100:
    print('\n'.join(lines[:30]))
    print(f'... (truncated, showing first 100 of {len(lines)} lines)')
else:
    print(output)

{
  "id": "2f93f0cf-cf42-4bde-889d-e3720c9246d6",
  "texts": [],
  "images": [
    {
      "width": 1008,
      "height": 644,
      "format": "image/png",
      "bit_depth": 32
    }
  ],
  "embeddings": {
    "float": [
      [
        -0.04345703,
        -0.053222656,
        0.0390625,
        0.009033203,
        0.00090789795,
        -0.048339844,
        0.010803223,
        -0.015380859,
        -0.024658203,
        0.033447266,
        0.036865234,
        0.05810547,
        0.030151367,
        -0.016723633,
        0.014709473,
        0.03515625,
... (truncated, showing first 100 of 1555 lines)


## Section 3) Using Embed 4 to generate embeddings for mixed modality input

In [3]:
import boto3
import json
import base64
import logging
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def get_base64_image_uri(image_file_path: str, image_mime_type: str):
    with open(image_file_path, "rb") as image_file:
        image_bytes = image_file.read()
        base64_image = base64.b64encode(image_bytes).decode("utf-8")
    return f"data:{image_mime_type};base64,{base64_image}"

def generate_embeddings(model_id, body, region_name):
    logger.info("Generating embeddings with Cohere Embed model %s", model_id)
    
    bedrock = boto3.client('bedrock-runtime', region_name=region_name)
    
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id,
        accept='*/*',
        contentType='application/json'
    )
    
    logger.info("Successfully generated embeddings")
    return response

def main():
    region_name = "us-west-2"
    image_file_path = "2025q2.png" 
    image_mime_type = "image/png"
    text = "Information throughout this presentation regarding the effect of foreign exchange rates, versus the U.S. Dollar, on our net sales and operating income is provided to show reported period operating results had the foreign exchange rates remained the same as those in effect in the comparable prior year periods."
    model_id = "us.cohere.embed-v4:0"
    image_base64_uri = get_base64_image_uri(image_file_path, image_mime_type)
    
    try:
        body = json.dumps({
            "inputs": [
                {
                  "content": [
                    {"type": "text", "text": text},
                    {"type": "image_url", "image_url": {"url": image_base64_uri}},
                  ]
                }
            ],
            "input_type": "search_document",
            "embedding_types": ["int8", "float"]
        })
        
        response = generate_embeddings(model_id, body, region_name)
        response_body = json.loads(response['body'].read())
        
        print(f"ID: {response_body['id']}")
        print(f"Response type: {response_body['response_type']}")
        print("Embeddings:")
        
        embeddings = response_body['embeddings']
        for embedding_type in embeddings:
            embedding_data = embeddings[embedding_type][0]
            sample_values = embedding_data[:5]  # Show first 5 values
            print(f"\t{embedding_type} Embeddings ({len(embedding_data)} dimensions):")
            print(f"\t\tSample values: {sample_values}")
        
        print("Multimodal embedding generated successfully")
        
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print(f"A client error occurred: {message}")
    except Exception as e:
        logger.error("An error occurred: %s", str(e))
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


INFO:__main__:Generating embeddings with Cohere Embed model us.cohere.embed-v4:0
INFO:__main__:Successfully generated embeddings


ID: e62d114d-a567-4aa3-977c-431836a1cf08
Response type: embeddings_by_type
Embeddings:
	float Embeddings (1536 dimensions):
		Sample values: [-0.041992188, -0.04272461, 0.045410156, 0.017211914, -0.0017852783]
	int8 Embeddings (1536 dimensions):
		Sample values: [-67, -68, 55, 15, -10]
Multimodal embedding generated successfully


## Section 4) Create S3 Vector Bucket and Index

In [4]:
import boto3
import json
import uuid
from datetime import datetime

# Create S3 and S3 Vectors clients
s3 = boto3.client("s3", region_name="us-west-2")
s3vectors = boto3.client("s3vectors", region_name="us-west-2")

date_str = datetime.now().strftime("%Y%m%d")
unique_id = str(uuid.uuid4())[:8]
bucket_name = f"my-s3-vector-bucket-{date_str}-{unique_id}"
index_name = f"my-s3-vector-index-1536-{date_str}-{unique_id}"

# Create S3 bucket
try:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': 'us-west-2'}
    )
    print(f"Created bucket: {bucket_name}")
except Exception as e:
    print(f"Bucket creation failed or already exists: {e}")

# Create vector bucket
try:
    s3vectors.create_vector_bucket(vectorBucketName=bucket_name)
    print(f"Created vector bucket: {bucket_name}")
except Exception as e:
    print(f"Vector bucket creation failed or already exists: {e}")

# Create vector index
s3vectors.create_index(
    vectorBucketName=bucket_name,
    indexName=index_name,
    dimension=1536,  # Cohere Embed v4 actual dimension
    dataType="float32",
    distanceMetric="cosine"
)

print(f"Created vector index: {index_name}")


Created bucket: my-s3-vector-bucket-20251025-bfa0a88d
Created vector bucket: my-s3-vector-bucket-20251025-bfa0a88d
Created vector index: my-s3-vector-index-1536-20251025-bfa0a88d


## Section 5) Store embeddings in S3 Vector Bucket Index

In [5]:
# Populate a vector index with embeddings from Cohere Embed V4 - Finance Example
import boto3
import json

# Create Bedrock Runtime and S3 Vectors clients in the AWS Region of your choice. 
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")
s3vectors = boto3.client("s3vectors", region_name="us-west-2")

# Financial documents to convert to embeddings.
texts = [
    "Q3 2024 earnings report shows revenue growth of 15% year-over-year driven by strong performance in cloud services and AI products",
    "Federal Reserve announces 0.25% interest rate cut to stimulate economic growth amid concerns about inflation and employment rates", 
    "ESG investment strategy focuses on sustainable companies with strong environmental policies and social responsibility initiatives",
    "Risk management framework includes stress testing, credit risk assessment, and regulatory compliance monitoring for financial institutions",
    "Cryptocurrency market volatility impacts institutional investment portfolios with Bitcoin and Ethereum showing significant price fluctuations",
    "Merger and acquisition activity in the technology sector reaches $150 billion as companies consolidate to compete in AI and cloud computing",
    # French 
    "Les résultats du T2 2024 révèlent une hausse du chiffre d'affaires de 18% portée par l'expansion des services de cybersécurité et blockchain",
    "La Banque centrale européenne maintient ses taux directeurs à 4% pour contrôler l'inflation tout en soutenant la reprise économique",
    "Les fonds d'investissement durable privilégient les entreprises à faible empreinte carbone et gouvernance transparente",
    "Le cadre de gestion des risques intègre l'analyse prédictive, l'évaluation du risque opérationnel et la surveillance réglementaire",
    "La volatilité des actifs numériques affecte les portefeuilles institutionnels avec des variations importantes sur Solana et Cardano",
    "Les opérations de fusion-acquisition dans le secteur fintech atteignent 85 milliards d'euros avec une consolidation accrue",
    # German
    "Halbjahresbericht 2024 verzeichnet 12% Gewinnsteigerung durch erfolgreiche Digitalisierung und Automatisierungslösungen",
    "Europäische Zentralbank senkt Leitzins um 0,5% zur Ankurbelung der Konjunktur bei stabiler Inflationsrate",
    "Nachhaltige Anlagestrategie bevorzugt Unternehmen mit innovativen Umwelttechnologien und ethischen Geschäftspraktiken",
    "Risikomanagement-System umfasst Liquiditätsprüfung, Marktrisikobewertung und kontinuierliche Compliance-Überwachung",
    "Kryptowährungsmarkt-Schwankungen beeinflussen institutionelle Anlageportfolios mit erheblichen Preisbewegungen bei Polygon und Avalanche",
    "Fusionsaktivitäten im Energiesektor erreichen 120 Milliarden Dollar bei verstärkter Konsolidierung erneuerbarer Technologien",
    # Chinese 
    "2024年上半年财务报告显示净利润增长20%，主要受益于金融科技和数字支付业务的快速发展",
    "中国人民银行调整存款准备金率0.5%以维持市场流动性并支持实体经济发展",
    "绿色投资基金重点关注清洁能源企业和具有创新环保技术的上市公司",
    "风险控制体系包括实时监控、信用风险评估和合规性审查以确保金融机构稳健运营",
    "数字资产市场波动影响机构投资组合，其中莱特币和瑞波币出现显著价格变动",
    "医疗健康行业并购交易达到800亿人民币，企业通过整合资源提升市场竞争力"
]

# Generate vector embeddings using correct Cohere Embed v4 format.
response = bedrock_runtime.invoke_model(
    modelId=inference_profile_id,
    # modelId="cohere.embed-v4:0",
    body=json.dumps({
        "texts": texts,
        "input_type": "search_document",
        "embedding_types": ["float"]
    }),
    accept='*/*',
    contentType='application/json'
)

# Extract embeddings from response.
response_body = json.loads(response["body"].read())
embeddings = response_body["embeddings"]["float"]

# Write embeddings into vector index with metadata for all languages.
vectors = []
for i, text in enumerate(texts):
    lang = "en" if i < 6 else ("fr" if i < 12 else ("de" if i < 18 else "zh"))
    vectors.append({
        "key": f"doc_{i}_{lang}",
        "data": {"float32": embeddings[i]},
        "metadata": {"source_text": text, "language": lang, "doc_id": i}
    })

s3vectors.put_vectors(
    vectorBucketName=bucket_name,   
    indexName=index_name,
    vectors=vectors
)

print(f"Successfully stored {len(texts)} multilingual financial document embeddings in vector index!")

Successfully stored 24 multilingual financial document embeddings in vector index!


## Section 6) Query S3 Vectors using a Strands Agent

In [6]:
import boto3
import json
from strands import Agent
from strands.tools import tool
import uuid

# S3 Vector search function for financial documents
@tool
def search(query_text: str, bucket_name: str = bucket_name, 
           index_name: str = index_name, top_k: int = 3, 
           category_filter: str = None) -> str:
    """Search financial documents using semantic vector search"""
    
    bedrock = boto3.client("bedrock-runtime", region_name="us-west-2")
    s3vectors = boto3.client("s3vectors", region_name="us-west-2")
    
    # Generate embedding using Cohere Embed v4
    response = bedrock.invoke_model(
        modelId=inference_profile_id,
        # modelId="cohere.embed-v4:0",
        body=json.dumps({
            "texts": [query_text],
            "input_type": "search_query",
            "embedding_types": ["float"]
        }),
        accept='*/*',
        contentType='application/json'
    )
    
    response_body = json.loads(response["body"].read())
    embedding = response_body["embeddings"]["float"][0]
    
    # Query vectors
    query_params = {
        "vectorBucketName": bucket_name,
        "indexName": index_name,
        "queryVector": {"float32": embedding},
        "topK": top_k,
        "returnDistance": True,
        "returnMetadata": True
    }
    
    if category_filter:
        query_params["filter"] = {"category": category_filter}
    
    response = s3vectors.query_vectors(**query_params)
    return json.dumps(response["vectors"], indent=2)\

# Create financial research agent using Strands
agent = Agent(
    name=f"Agent_{uuid.uuid4().hex[:8]}",
    system_prompt="You are a financial research assistant that can search through financial documents, earnings reports, regulatory filings, and market analysis. Use the search tool to find relevant financial information and provide helpful analysis.",
    tools=[search]
)


# Example usage
if __name__ == "__main__":
    print("=== Strands Agent Financial Search Examples ===\n")
    
    print("1. Direct Tool Call:")
    result = search("Compare earnings growth rates mentioned in the documents")
    print(result)
    
    print("\n2. Agent Query:")
    response = agent("Compare earnings growth rates mentioned in the documents")
    print(response)

# Other Questions
# "What are the different central bank policies across regions?"
# "Compare earnings growth rates mentioned in the documents"
# "What investment strategies are discussed across different markets?"
# "How do risk management approaches vary by region?"
# "What cryptocurrency trends are mentioned globally?"
# "Compare merger and acquisition activities across sectors"

=== Strands Agent Financial Search Examples ===

1. Direct Tool Call:


INFO:strands.telemetry.metrics:Creating Strands MetricsClient


[
  {
    "key": "doc_0_en",
    "metadata": {
      "doc_id": 0,
      "language": "en",
      "source_text": "Q3 2024 earnings report shows revenue growth of 15% year-over-year driven by strong performance in cloud services and AI products"
    },
    "distance": 0.7292724251747131
  },
  {
    "key": "doc_18_zh",
    "metadata": {
      "doc_id": 18,
      "source_text": "2024\u5e74\u4e0a\u534a\u5e74\u8d22\u52a1\u62a5\u544a\u663e\u793a\u51c0\u5229\u6da6\u589e\u957f20%\uff0c\u4e3b\u8981\u53d7\u76ca\u4e8e\u91d1\u878d\u79d1\u6280\u548c\u6570\u5b57\u652f\u4ed8\u4e1a\u52a1\u7684\u5feb\u901f\u53d1\u5c55",
      "language": "zh"
    },
    "distance": 0.7704182863235474
  },
  {
    "key": "doc_12_de",
    "metadata": {
      "source_text": "Halbjahresbericht 2024 verzeichnet 12% Gewinnsteigerung durch erfolgreiche Digitalisierung und Automatisierungsl\u00f6sungen",
      "doc_id": 12,
      "language": "de"
    },
    "distance": 0.7738275527954102
  }
]

2. Agent Query:
I'll search for i

## Section 7) Deploy Strands Agent locally

In [7]:
# Install the AgentCore starter toolkit
!pip install -q bedrock-agentcore-starter-toolkit

In [8]:
!pip install --force-reinstall -U -r requirements.txt --q

# This example pins the following versions: bedrock-agentcore<=0.1.5 bedrock-agentcore-starter-toolkit==0.1.14

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.59 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.55.2 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0,

In [9]:
%%writefile example.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
import boto3
import os
import uuid
from strands.models import BedrockModel

# Configuration from environment variables with defaults
REGION = os.getenv('AWS_REGION', 'us-west-2')
EMBED_MODEL_ID = os.getenv('EMBED_MODEL_ID', 'us.cohere.embed-v4:0')
AGENT_MODEL_ID = os.getenv('AGENT_MODEL_ID', 'us.anthropic.claude-3-7-sonnet-20250219-v1:0')

@tool
def search(query_text: str, bucket_name: str = None, 
           index_name: str = None, top_k: int = 3, 
           category_filter: str = None) -> str:
    """Search financial documents using semantic vector search"""
    
    bucket_name = ''
    index_name = ''
    
    bedrock = boto3.client("bedrock-runtime", region_name=REGION)
    s3vectors = boto3.client("s3vectors", region_name=REGION)
    
    response = bedrock.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=json.dumps({
            "texts": [query_text],
            "input_type": "search_query",
            "embedding_types": ["float"]
        }),
        accept='*/*',
        contentType='application/json'
    )
    
    response_body = json.loads(response["body"].read())
    embedding = response_body["embeddings"]["float"][0]
    
    query_params = {
        "vectorBucketName": bucket_name,
        "indexName": index_name,
        "queryVector": {"float32": embedding},
        "topK": top_k,
        "returnDistance": True,
        "returnMetadata": True
    }
    
    if category_filter:
        query_params["filter"] = {"category": category_filter}
    
    response = s3vectors.query_vectors(**query_params)
    return json.dumps(response["vectors"], indent=2)

model = BedrockModel(model_id=AGENT_MODEL_ID)
agent = Agent(
    name=f"Agent_{uuid.uuid4().hex[:8]}",
    model=model,
    tools=[search],
    system_prompt="You are a financial research assistant that can search through financial documents, earnings reports, regulatory filings, and market analysis. Use the search tool to find relevant financial information and provide helpful analysis."
)

def strands_agent_bedrock(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))


Overwriting example.py


In [10]:
# Update example.py with actual bucket and index names
with open('example.py', 'r') as f:
    content = f.read()

# Replace the empty strings with actual values
content = content.replace(
    "bucket_name = ''",
    f"bucket_name = '{bucket_name}'"
)
content = content.replace(
    "index_name = ''",
    f"index_name = '{index_name}'"
)

with open('example.py', 'w') as f:
    f.write(content)

print(f"Updated example.py with bucket: {bucket_name} and index: {index_name}")

Updated example.py with bucket: my-s3-vector-bucket-20251025-bfa0a88d and index: my-s3-vector-index-1536-20251025-bfa0a88d


In [11]:
# Invoke agent locally
!python example.py '{"prompt": "Compare earnings growth rates mentioned in the documents"}'

I'd be happy to compare earnings growth rates mentioned in the financial documents. To provide you with accurate information, I'll need to search for relevant content about earnings growth rates.
Tool #1: search
Let me search for more specific information about earnings growth rates across different periods or companies:
Tool #2: search
Let me search for one more set of results to ensure comprehensive coverage:
Tool #3: search
# Comparison of Earnings Growth Rates

Based on my search through the financial documents, I can provide a comparison of the various earnings growth rates mentioned:

| Report Period | Growth Rate | Growth Type | Key Drivers | Language of Source |
|---------------|------------|-------------|------------|-------------------|
| Q3 2024 | 15% | Revenue growth (year-over-year) | Strong performance in cloud services and AI products | English |
| H1 2024 | 20% | Net profit growth | Rapid development of fintech and digital payment services | Chinese |
| H1 2024 | 12% | 

## Section 8) Deploy Strands Agent on Amazon Bedrock AgentCore

In [12]:
%%writefile example.py
from strands import Agent, tool
from strands_tools import calculator
import json
import boto3
import os
import uuid
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Configuration from environment variables with defaults
REGION = os.getenv('AWS_REGION', 'us-west-2')
EMBED_MODEL_ID = os.getenv('EMBED_MODEL_ID', 'us.cohere.embed-v4:0')
AGENT_MODEL_ID = os.getenv('AGENT_MODEL_ID', 'us.anthropic.claude-3-7-sonnet-20250219-v1:0')

@tool
def search(query_text: str, bucket_name: str = None, 
           index_name: str = None, top_k: int = 3, 
           category_filter: str = None) -> str:
    """Search financial documents using semantic vector search"""
    
    bucket_name = ''
    index_name = ''
    
    bedrock = boto3.client("bedrock-runtime", region_name=REGION)
    s3vectors = boto3.client("s3vectors", region_name=REGION)
    
    response = bedrock.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=json.dumps({
            "texts": [query_text],
            "input_type": "search_query",
            "embedding_types": ["float"]
        }),
        accept='*/*',
        contentType='application/json'
    )
    
    response_body = json.loads(response["body"].read())
    embedding = response_body["embeddings"]["float"][0]
    
    query_params = {
        "vectorBucketName": bucket_name,
        "indexName": index_name,
        "queryVector": {"float32": embedding},
        "topK": top_k,
        "returnDistance": True,
        "returnMetadata": True
    }
    
    if category_filter:
        query_params["filter"] = {"category": category_filter}
    
    response = s3vectors.query_vectors(**query_params)
    return json.dumps(response["vectors"], indent=2)

model = BedrockModel(model_id=AGENT_MODEL_ID)
agent = Agent(
    name=f"Agent_{uuid.uuid4().hex[:8]}",
    model=model,
    tools=[search],
    system_prompt="You are a financial research assistant that can search through financial documents, earnings reports, regulatory filings, and market analysis. Use the search tool to find relevant financial information and provide helpful analysis."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()


Overwriting example.py


In [13]:
# Update example.py with actual bucket and index names
with open('example.py', 'r') as f:
    content = f.read()

# Replace the empty strings with actual values
content = content.replace(
    "bucket_name = ''",
    f"bucket_name = '{bucket_name}'"
)
content = content.replace(
    "index_name = ''",
    f"index_name = '{index_name}'"
)

with open('example.py', 'w') as f:
    f.write(content)

print(f"Updated example.py with bucket: {bucket_name} and index: {index_name}")

Updated example.py with bucket: my-s3-vector-bucket-20251025-bfa0a88d and index: my-s3-vector-index-1536-20251025-bfa0a88d


In [14]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import uuid

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = f"search_agent_{uuid.uuid4().hex[:8]}"
response = agentcore_runtime.configure(
    entrypoint="example.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response


Entrypoint parsed: file=/home/sagemaker-user/20251024working_example/example.py, bedrock_agentcore_name=example
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/sagemaker-user/20251024working_example/example.py, bedrock_agentcore_name=example
Configuring BedrockAgentCore agent: search_agent_95b134e7
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: search_agent_95b134e7


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/sagemaker-user/20251024working_example/Dockerfile
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated Dockerfile: /home/sagemaker-user/20251024working_example/Dockerfile
Generated .dockerignore: /home/sagemaker-user/20251024working_example/.dockerignore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated .dockerignore: /home/sagemaker-user/20251024working_example/.dockerignore
Changing default agent from 'search_agent_6e2dbe2d' to 'search_agent_95b134e7'
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.config:Changing default agent from 'search_agent_6e2dbe2d' to 'search_agent_95b134e7'
Bedrock AgentCore configured: /home/sagemaker-user/20251024working_example/.bedrock_agentcore.yaml
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Bedrock AgentCore configured: /home/sagemaker-user/20251024working_example/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/sagemaker-user/20251024working_example/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/sagemaker-user/20251024working_example/Dockerfile'), dockerignore_path=PosixPath('/home/sagemaker-user/20251024working_example/.dockerignore'), runtime='None', region='us-west-2', account_id='809719347864', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [15]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • No local Docker required
💡 Available deployment modes:
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentc

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-search_agent_95b134e7


✅ ECR repository available: 809719347864.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-search_agent_95b134e7
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ ECR repository available: 809719347864.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-search_agent_95b134e7
Getting or creating execution role for agent: search_agent_95b134e7
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Getting or creating execution role for agent: search_agent_95b134e7
Using AWS region: us-west-2, account ID: 809719347864
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Using AWS region: us-west-2, account ID: 809719347864
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-053c7ca94f
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-053c7ca94f
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-053c7ca94f
INFO:bedrock_agentcore_starter_toolki

In [16]:
# Check the AgentCore status
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: search_agent_95b134e7
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: search_agent_95b134e7


'READY'

In [17]:
import boto3
import json

iam = boto3.client('iam')
session = boto3.Session()
region = session.region_name

# Find AgentCore runtime role dynamically
roles = iam.list_roles()['Roles']
role_name = next(
    role['RoleName'] for role in roles 
    if role['RoleName'].startswith(f'AmazonBedrockAgentCoreSDKRuntime-{region}-')
)

policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "s3vectors:QueryVectors",
                "s3vectors:PutVectors",
                "s3vectors:GetVectors"
            ],
            "Resource": "*"
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName='S3VectorBedrockAccess',
        PolicyDocument=json.dumps(policy_document)
    )
    print(f"✅ Added S3 Vector and Bedrock permissions to AgentCore role: {role_name}")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Added S3 Vector and Bedrock permissions to AgentCore role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-053c7ca94f


In [18]:
invoke_response = agentcore_runtime.invoke({"prompt": "Compare earnings growth rates mentioned in the documents"})

invoke_response

{'ResponseMetadata': {'RequestId': 'f722109b-bf77-48d9-bc49-28f387c591da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 25 Oct 2025 05:38:24 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f722109b-bf77-48d9-bc49-28f387c591da',
   'baggage': 'Self=1-68fc6240-257f940b77e27234299de73d,session.id=54c56e63-0444-464e-8042-e63e2ed120d9',
   'x-amzn-bedrock-agentcore-runtime-session-id': '54c56e63-0444-464e-8042-e63e2ed120d9',
   'x-amzn-trace-id': 'Root=1-68fc6240-0f9b002707c0225e7fb67cb6;Parent=90ffda2745f4e8e0;Sampled=1;Self=1-68fc6240-257f940b77e27234299de73d'},
  'RetryAttempts': 0},
 'runtimeSessionId': '54c56e63-0444-464e-8042-e63e2ed120d9',
 'traceId': 'Root=1-68fc6240-0f9b002707c0225e7fb67cb6;Parent=90ffda2745f4e8e0;Sampled=1;Self=1-68fc6240-257f940b77e27234299de73d',
 'baggage': 'Self=1-68fc6240-257f940b77e27234299de73d,session.id=54c56e63-0444-464e-8042-e63e2ed120d9',
 'contentType': 

## Section 9) Cleanup

Run this code to delete the resources created in this Jupyter Notebook.

In [19]:
def cleanup_all_resources():
    """Delete all resources created during the notebook execution"""
    
    # Initialize clients
    agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name='us-west-2')
    ecr_client = boto3.client('ecr', region_name='us-west-2')
    s3_client = boto3.client('s3')
    s3vectors_client = boto3.client('s3vectors', region_name='us-west-2')
    iam_client = boto3.client('iam')
    codebuild_client = boto3.client('codebuild', region_name='us-west-2')
    
    # 1. Delete AgentCore Runtime
    try:
        print("Deleting AgentCore Runtime...")
        agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
        print(f"✅ AgentCore Runtime {launch_result.agent_id} deleted")
    except Exception as e:
        print(f"❌ AgentCore Runtime: {str(e)}")
    
    # 2. Delete ECR Repository
    try:
        print("Deleting ECR Repository...")
        ecr_client.delete_repository(repositoryName=launch_result.ecr_uri.split('/')[1], force=True)
        print(f"✅ ECR Repository deleted")
    except Exception as e:
        print(f"❌ ECR Repository: {str(e)}")
    
    # 3. Delete S3 Vector Index and Bucket
    try:
        print("Deleting S3 Vector Index...")
        s3vectors_client.delete_index(vectorBucketName=bucket_name, indexName=index_name)
        print(f"✅ S3 Vector Index {index_name} deleted")
    except Exception as e:
        print(f"❌ S3 Vector Index: {str(e)}")
    
    try:
        print("Deleting S3 Vector Bucket...")
        s3vectors_client.delete_vector_bucket(vectorBucketName=bucket_name)
        print(f"✅ S3 Vector Bucket {bucket_name} deleted")
    except Exception as e:
        print(f"❌ S3 Vector Bucket: {str(e)}")
    
    try:
        print("Deleting S3 Bucket...")
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"✅ S3 Bucket {bucket_name} deleted")
    except Exception as e:
        print(f"❌ S3 Bucket: {str(e)}")
    
    # 4. Delete CodeBuild Project
    try:
        print("Deleting CodeBuild Project...")
        projects = codebuild_client.list_projects()['projects']
        agent_project = next((p for p in projects if agent_name in p), None)
        if agent_project:
            codebuild_client.delete_project(name=agent_project)
            print(f"✅ CodeBuild Project {agent_project} deleted")
    except Exception as e:
        print(f"❌ CodeBuild Project: {str(e)}")
    
    # 5. Delete IAM Roles
    try:
        print("Deleting IAM Roles...")
        roles = iam_client.list_roles()['Roles']
        
        # Delete AgentCore runtime role
        runtime_role = next((r['RoleName'] for r in roles if 'AmazonBedrockAgentCoreSDKRuntime' in r['RoleName']), None)
        if runtime_role:
            # Delete inline policies
            policies = iam_client.list_role_policies(RoleName=runtime_role)
            for policy_name in policies['PolicyNames']:
                iam_client.delete_role_policy(RoleName=runtime_role, PolicyName=policy_name)
            iam_client.delete_role(RoleName=runtime_role)
            print(f"✅ Runtime IAM Role {runtime_role} deleted")
        
        # Delete CodeBuild role
        codebuild_role = next((r['RoleName'] for r in roles if 'AmazonBedrockAgentCoreSDKCodeBuild' in r['RoleName']), None)
        if codebuild_role:
            # Delete inline policies
            policies = iam_client.list_role_policies(RoleName=codebuild_role)
            for policy_name in policies['PolicyNames']:
                iam_client.delete_role_policy(RoleName=codebuild_role, PolicyName=policy_name)
            iam_client.delete_role(RoleName=codebuild_role)
            print(f"✅ CodeBuild IAM Role {codebuild_role} deleted")
            
    except Exception as e:
        print(f"❌ IAM cleanup: {str(e)}")
    
    print("\n🎉 Complete cleanup finished!")

# Run cleanup
cleanup_all_resources()


Deleting AgentCore Runtime...
✅ AgentCore Runtime search_agent_95b134e7-f3xz7Q5XwV deleted
Deleting ECR Repository...
✅ ECR Repository deleted
Deleting S3 Vector Index...
✅ S3 Vector Index my-s3-vector-index-1536-20251025-bfa0a88d deleted
Deleting S3 Vector Bucket...
✅ S3 Vector Bucket my-s3-vector-bucket-20251025-bfa0a88d deleted
Deleting S3 Bucket...
✅ S3 Bucket my-s3-vector-bucket-20251025-bfa0a88d deleted
Deleting CodeBuild Project...
✅ CodeBuild Project bedrock-agentcore-search_agent_95b134e7-builder deleted
Deleting IAM Roles...
✅ Runtime IAM Role AmazonBedrockAgentCoreSDKRuntime-us-west-2-053c7ca94f deleted
✅ CodeBuild IAM Role AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-053c7ca94f deleted

🎉 Complete cleanup finished!
